# Homework 1: Language Identification
11-411/11-611 Natural Language Processing (Fall 2024)

- RELEASED: Tuesday, Feb 4, 2025
- DUE: Tuesday, Feb 18, 2025 11:59 pm EDT

**Submission**: Please upload this single file named `HW1.ipynb` to [gradescope](https://www.gradescope.com/courses/557600). Ensure you do not rename the file prior to submission.


In this assignment, you will build a language identification classifier that distinguishes between six languages:

- [Hausa](https://en.wikipedia.org/wiki/Hausa_language)
- [Indonesian](https://en.wikipedia.org/wiki/Indonesian_language)
- [Manobo](https://en.wikipedia.org/wiki/Manobo_languages)
- [Nahuatl](https://en.wikipedia.org/wiki/Nahuatl)
- [Swahili](https://en.wikipedia.org/wiki/Swahili_language)
- [Tagalog](https://en.wikipedia.org/wiki/Tagalog_language)

Some languages can be distinguished easily, because they use different scripts. These six languages, however, use the same ([Latin](https://en.wikipedia.org/wiki/Latin_script)) script with minimal [diacritics](https://en.wikipedia.org/wiki/Diacritic) so it is difficult to hand-craft classifiers based on the presence or absence of particular characters. Indeed, unless you have linguistic training or familiarity with the languages, it is difficult to tell them apart.

How can they be distinguished? A naïve approach is to use word counts as unigram features. However, the number of possible words in a large corpus of five languages is vast. It is essential to look at something smaller — characters.

Even though the six languages use roughly the same characters, the relative frequencies of these characters vary greatly. Thus, using characters as features (unigram character models) is appealing (and fairly effective). It is also true that languages vary in their *phonotactics*, the way in which consonants and vowels combine in sequence. Thus, looking at character ngrams (for small values of $n$) is also appealing (and effective). Note, however, that as the value of $n$ increases, this approach runs into the same problem as the word unigram model (sparcity). In this scenario, the model is likely to overfit.

Various kinds of classifiers can be used for this application. NB classifiers, for example, are quite effective. However, inference is slow and performance, given the same training set, is likely to be worse than other options. Simple logistic regression cannot be used because this is an n-way (multinomial) classification problem. Multinomial Logistic Regression (Softmax Regression) is a good fit.

## Requirements

You will perform the following tasks:

1. Implement a training loop for Multinomial Logistic Regression.
2. Implement inference for Multinomal Logistic Regression
3. Determine the optimal order of $n$ for ngrams for MNLR trained on the training set.
4. Calculate and display a confusion matrix for a trigram model evaluated on the test set.
5. Inspect the feature weights, and display the most predictive features for each language.

## How to Use Jupyter Notebooks for Our Assignment
Throughout our first assignment, we'll utilize Jupyter Notebooks to walk you through concepts, let you implement them, and also allow you to experiment on your own. By the end of this assignment, you'll not only understand word embeddings more deeply but will also become familiar with a powerful tool used extensively in the data science community.

### Types of Cells:
**Markdown Cells**: These cells (like the one you're reading now) are utilized to write text, frame explanations, embed images, or even formulate equations. They make our notebook more explanatory and structured.

**Code Cells**: This is where the action happens. In these cells, you'll write and execute Python code. They will play a critical role in our exercises as you experiment with word embeddings.

*Warning*: Refrain from rearranging, adding, or deleting any cells.

### Runtime Volatility
As you navigate and execute the cells within this Jupyter notebook, it's crucial to understand that the runtime environment is volatile. In simpler terms:

*   If you restart the notebook or experience a disconnection, all your in-memory data and variables will be lost.
*   While the code and markdown cells will remain, the outputs from code cells will need to be regenerated by rerunning them.

Therefore, if you're working on a task over an extended period or with large datasets, remember to save your results and progress frequently to avoid potential data loss.


### Running this Notebook on Google Colab
Google Colab is a free Jupyter notebook environment that requires no setup and runs entirely in the cloud. To run this notebook on Colab:

1.   Save a copy of this notebook on your **Google Drive**.
2.   Open the notebook in Google Colab.
3.   In Google Colab, you can execute each cell using the play button (or use the keyboard shortcuts mentioned above).

**Tip**: Google Colab may automatically disconnect after a certain period of inactivity. Keep this in mind, especially when running longer tasks.

**Note**: Google Colab provides free access to GPUs and TPUs, which might be useful for later assignments.

## Imports
Do not change.

In [59]:
import csv
import random

import numpy as np
import numpy.typing as npt
import numpy.testing as testing

from collections import Counter

## Helper Functions

### Metrics for Binary Classification

In [60]:
def precision(tp: int, fp: int) -> float:
    """Computes the precision, given true positives and false positives."""
    return tp / (tp + fp)


def recall(tp: int, fn: int) -> float:
    """Computes the recall, given the true positives and false negatives."""
    return tp / (tp + fn)


def f_measure(beta: float, tp: int, fp: int, fn: int) -> float:
    """Computes the F-measure for a given beta, true positives, false positives, and false negatives."""
    return (
        (1 + beta**2)
        * (precision(tp, fp) * recall(tp, fn))
        / (beta**2 * precision(tp, fp) * recall(tp, fn))
    )


def f1(tp: int, fp: int, fn: int) -> float:
    """Computes the F1 measure for a given TP, FP, and FN."""
    return f_measure(1, tp, fp, fn)

### Micro-Averaged Metrics

In [61]:
def micro_precision(tp: "dict[str, int]", fp: "dict[str, int]") -> float:
    """Computes micro-averaged precision."""
    tp_sum = sum(tp.values())
    fp_sum = sum(fp.values())
    return tp_sum / (tp_sum + fp_sum)


def micro_recall(tp: "dict[str, int]", fn: "dict[str, int]") -> float:
    """Computes micro-averaged recall."""
    tp_sum = sum(tp.values())
    fn_sum = sum(fn.values())
    return tp_sum / (tp_sum + fn_sum)


def micro_f1(tp: "dict[str, int]", fp: "dict[str, int]", fn: "dict[str, int]") -> float:
    """Computes micro-averaged F1."""
    mp = micro_precision(tp, fp)
    mr = micro_recall(tp, fn)
    return 2 * (mp * mr) / (mp + mr)

### Macro-Averaged Metrics

In [62]:
def macro_precision(tp: "dict[str, int]", fp: "dict[str, int]") -> float:
    """Computes macro-averaged precision."""
    n = len(tp)
    return (1 / n) * sum([precision(tp[c], fp[c]) for c in tp.keys()])


def macro_recall(tp: "dict[str, int]", fn: "dict[str, int]") -> float:
    """Computes macro-averaged recall."""
    n = len(tp)
    return (1 / n) * sum([recall(tp[c], fn[c]) for c in tp.keys()])


def macro_f1(tp: "dict[str, int]", fp: "dict[str, int]", fn: "dict[str, int]") -> float:
    """Computes macro-averaged F1."""
    n = len(tp)
    return (
        2
        * (macro_precision(tp, fp) * macro_recall(tp, fn))
        / (macro_precision(tp, fp) + macro_recall(tp, fn))
    )

## Preprocess Data

### Loading data from files

We first need to load data from the provided TSV files. Each file is two columns, the language of the document and the document text, separated by a tab (`\t`) character. We load this data into a list of tuples, to maintain the coupling between each document and its label.

In [63]:
def load_data(data_filename: str) -> list[tuple[str, str]]:
    with open(data_filename) as fin:
        reader = csv.reader(fin, delimiter="\t")
        language_document_tuples = [(lang, doc) for (lang, doc) in reader]
    return language_document_tuples

### Feature Extraction

We will use ngrams as features, so we need to be able to extract them.

In [64]:
def extract_ngrams(x: str, n=3) -> "list[str]":
    """Given a string, return all character ngrams of order `n`."""
    return ["".join(s) for s in (zip(*[x[i:] for i in range(n)]))]

### Language Codes to One-Hot Vectors
And we need a function to convert a language code into a **one-hot vector** (called $\mathbf{y}$).

In [65]:
def to_onehot_vector(lang: str, langs: list[str]) -> np.ndarray:
    y = np.zeros(len(langs))
    y[langs.index(lang)] = 1
    return y

We need to be able to convert `dict`s of ngram counts to vectors of ngram counts (using a map from ngrams to dimensions of the vectors).

In [66]:
def vectorize_ngrams(counter: dict[str, int], feature_map: dict[str, int]) -> np.ndarray:
    """
    Given a dict of ngram counts and a map from features to indices, returns a vector of ngram counts.

    Parameters
    ----------
    counter : dict
        Counter/dict of ngram counts
    feature_map : dict
        Map from ngrams to indices

    Returns
    -------
    np.ndarray
        A vector of ngram counts
    """
    feature_vector = np.zeros(len(feature_map))
    for ngram, count in counter.items():
        if ngram in feature_map:
            feature_vector[feature_map[ngram]] = count
    return feature_vector

And putting together the conversion from text into ngrams, and vectorizing the ngrams:

In [67]:
def vectorize_document(document: str, feature_map: dict[str, int], ngram_length: int) -> np.ndarray:
    document_ngrams = extract_ngrams(document, ngram_length)
    vector = vectorize_ngrams(Counter(document_ngrams), feature_map)
    return vector

We need separate functions for preprocessing the training observations and the dev/test observations. The former function must return a map from language names to labels as one-hot vectors as well as a map from features (ngrams) to indices of vector dimensions.

In [68]:
def preprocess_training_observations(
    training_observations: list[tuple[str, str]], n: int = 1
) -> tuple[list[tuple[np.ndarray, np.ndarray]], dict[str, np.ndarray], dict[str, int]]:
    langs = set()
    features = set()
    obs = []

    for lang, doc in training_observations:
        langs.add(lang)
        ngrams = extract_ngrams(doc, n)
        features = features | set(ngrams)
        obs.append((lang, ngrams))
    feature_map = {feature: idx for idx, feature in enumerate(sorted(features))}
    lang_list = list(sorted(langs))
    lang_map = {lang: to_onehot_vector(lang, lang_list) for lang in lang_list}

    obs = [
        (lang_map[lang], vectorize_ngrams(Counter(ngrams), feature_map)) for (lang, ngrams) in obs
    ]

    print(f"{len(obs)} training observations.")
    return obs, lang_map, feature_map

The function for preprocessing test observations (and dev observations) takes the feature map and the language map as arguments and returns only the list of labeled observations.

In [69]:
def preprocess_test_observations(
    test_observations, feature_map: dict[str, int], lang_map: dict[str, np.ndarray], n: int = 1
) -> tuple[list[np.ndarray], list[np.ndarray]]:
    obs = []

    for i, (lang, doc) in enumerate(test_observations):
        vectorized_doc = vectorize_document(doc, feature_map, ngram_length=n)
        try:
            obs.append((lang_map[lang], vectorized_doc))
        except KeyError:
            print(f"Unkown language {lang} at index {i}. Known languages are: {lang_map.keys()}")
    print(f"{len(obs)} test observations.")
    return obs

### Classification Function

We also need to define the softmax function.

Softmax is technically

$$\text{softmax}(z_i) = \frac{e^{z_i}}{\sum^K_{j=1} e^{z_j}}$$

However, if implemented naïvely, this is not numerically stable. Instead, we use:

$$\text{softmax}(z_i) = \frac{e^{z_i - \text{max}(z)}}{\sum^K_{j=1} e^{z_j - \text{max}(z)}}$$


In [70]:
def softmax(z: npt.ArrayLike) -> npt.ArrayLike:
    """Compute the softmax of a vector `z`"""
    # exp(z) can get very large. For numerical stability, we subtract a vector of very large values (np.max(z)) from z.
    return np.exp(z - np.max(z)) / np.exp(z - np.max(z)).sum()

## Training the Classifier

### Compute the gradient

The formula for computing one element in our gradient is as follows (the partial derivitive of the negative log likelihood loss):

$$\frac{\partial L_{CE}}{\partial \mathbf{w}_{k,i}}=-(\mathbf{y}_k-\hat{\mathrm{y}}_k)\mathbf{x}_i$$

where $k$ is the **class** (rows of the matrix $\mathbf{w}$) and $i$ corresponds the the feature (columns of the matrix $\mathbf{w}$).

We will define a function `grad` for computing the whole gradient, a $K \times N$ matrix.

**This is the first piece of code that you'll write.**

In [71]:
def grad(W: np.ndarray, y: np.ndarray, x: np.ndarray) -> np.ndarray:
    """
    Caculates the gradient of the negative log liklihood loss, a [K * N] matrix, with respect to W.

    Parameters
    ----------
    W : np.ndarray
        A matrix of of weights, expressed as a [K * N] matrix where K is the number of classes and N is the number of features.
    y : np.ndarray
       The true label of the observation, expressed as a one-hot encoded vector of shape [K].
    x : np.ndarray
        A vector of features.

    Returns
    -------
    np.ndarray
        The gradient of the loss with respect to W.
    """
    x = np.insert(x,0,[1])
    # biasCol = np.array([])
    #get the first col of W.
    # for i in range(0,len(W)):
    #   biasCol = np.append(biasCol, W[i][0])
    # biasCol = biasCol.reshape(-1, 1)
    sx = softmax(np.dot(W,x))
    # sx = sx.T
    # sx = sx.flatten()
    #print(sx, "this is the softmax")
    #print(np.subtract(y,sx), "one-hot - softmax")
    lossY = np.subtract(y,sx)
    # print(lossY)
    gradient = np.tensordot(lossY, x, 0) * -1
    # gradientFull = np.array([[]])
    # for k in range(0, len(y)):
    #   gradientPartial = []
    #   for i in range(0,len(x)):
    #     gradientElt = -1 * (np.subtract(y[k],sx[k])) * x[i]
    #     gradientPartial = np.append(gradientPartial, gradientElt)
    #   #print(gradientPartial, " yeah, this is going in my gradient")
    #   gradientFull = np.append(gradientFull, gradientPartial)
    # gradient = np.reshape(gradientFull, (-1, len(x)))
    #tensor add/subtract
    #How do we know what to update in W? What ARE we updating?
    #print(gradient, " this is the gradient")
    return gradient

### Training Loop

Iterate over the observations in the training set $n$ times (in random order). For each item, compute the one-hot vector $\mathbf{y}$ and the probability distribution $\hat{\mathbf{y}}$. Use these values to compute the gradient. Update the parameters based on the gradient. At the end of each epoch (pass through the training data), compute the true positives, false positives, and false negatives for each target class based on the current weights, and report micro-averaged precision and recall.

**Note**: The way that we evaluated the training loop depended on an in-place shuffle of the training data, which, while not technically wrong, is maybe not the most intuitive way to do it. When shuffling your data, please either use a shuffling method that does not change the observations parameter in-place. In particular, please use `random.sample` like:
```
shuffled_observations = random.sample(observations, len(observations))
```

How you report the metrics is up to you — we will not look at what you output to STDOUT — but it is important that you do this. **Otherwise, you will not be able to determine whether your model is training.**

You can also output the loss at each step (very noisy!), each epoch, or run the classifier on the dev set and report the metrics at the end of each epoch.

Remember that the 0th column in $W$ contains the biases. You will have to insert a $1$ at the beginning of the feature vector $x$ in order to accomodate this.

In [72]:
def train(observations: tuple[np.ndarray, np.ndarray], eta: float, epochs: int = 1) -> np.ndarray:
    """
    Give a set of observations, returns a trained multinomial LR (softmax regression) model.

    Parameters
    ----------
    observations : (np.ndarray, np.ndarray)
        Pairs consisting of a tuple of NumPy arrays (a one-hot vector encoding the ground truth language labels and a vector of features)
    epochs : int
        The number of epochs to train the model.

    Returns
    -------
    np.ndarray
        The moodel as a [K * N] weight matrix.
    """
    tp = {"hausa": 0, "indonesian": 0, "manobo": 0, "nahuatl": 0,"swahili": 0, "tagalog": 0}
    #not yet updated
    fp = {"hausa": 0, "indonesian": 0, "manobo": 0, "nahuatl": 0,"swahili": 0, "tagalog": 0}
    fn = {"hausa": 0, "indonesian": 0, "manobo": 0, "nahuatl": 0,"swahili": 0, "tagalog": 0}
    idxToName = ["hausa", "indonesian", "manobo", "nahuatl", "swahili", "tagalog"]
    #These are X by 1 vectors
    #print(observations[0])
    #These are X by 1 vectors, but these need to be 1 by X vectors.
    #print(observations[1])
    #The amount of classes is the length of the one hot vector of labels.
    K = len(observations[0][0])
    #The number of features is 1 + the length of th feature vector (account for biases)
    N = len(observations[0][1]) + 1
    W = np.zeros((K,N))
    #print(W)
    for ep in range(0,epochs):
      obsLen = len(observations)
      shuffled_observations = random.sample(observations, obsLen)
      for i in range(0,obsLen):
        featuresRow = shuffled_observations[i][1]
        features = featuresRow.reshape(-1, 1)
        oneHot = shuffled_observations[i][0]
        #print(features)
        #print(oneHot)
        if oneHot[classify(W, features)] == 1:
          #can we rely on order being as stated in the writeup?
          #need to get from id to string somehow
          tp[idxToName[classify(W, features)]] += 1
        else:
          fp[idxToName[classify(W, features)]] += 1
          #find the index of the 1 inthe one hot vector
          max = 0
          mxidx = -1
          for j in range(0,len(oneHot)):
            if oneHot[j] > max:
              max = oneHot[j]
              mxidx = j
          fn[idxToName[mxidx]] += 1
          #I guess ill insert the 0 to account for the biases here? Do i need to use this above?
        gradi = grad(W, oneHot, features)
        W = np.subtract(W,(eta * gradi))
        #print(W)
      mp = micro_precision(tp, fp)
      mr = micro_recall(tp, fn)
      print(mp, " is the micro precision \n", mr, "is the micro recall")
    return W

## Classification

The classification function is very simple.

In [73]:
def classify(W: np.ndarray, x: np.ndarray) -> np.intp:
    """
    Return the index of the hypothesized language.

    Parameters
    ----------
    W : np.ndarray
        Weight matrix (one row for each category/language, on column for each feature)
    x : np.ndarray
        Vector of real-valuled features

    Returns
    -------
    intp
        The index of the hypothesized language.
    """
   # biasCol = np.array([])
    x = np.insert(x,0,[1])
    #get the first col of W.
    #print(len(W))
    # for i in range(0,len(W)):
    #   biasCol = np.append(biasCol, W[i][0])
    #print(np.dot(W,x), " This is the feature vector when product with the weight matrix")
    #print(biasCol, " These are our biases")
    # biasCol = biasCol.reshape(-1, 1)
    sx = softmax(np.dot(W,x))
    #print(sx)
    # max = 0
    # mxidx = -1
    mxidx = np.argmax(sx)
    # for i in range(0,len(sx)):
    #   if sx[i][0] > max:
    #     max = sx[i][0]
    #     mxidx = i
    #print(mxidx)
    return mxidx

## Evaluate the Classifier

Then, a function to train and evaluate the classifier.

In [74]:
def evaluate(train_set, test_set, eta, epochs=3):
    """Trains and evaluates a model."""
    print("Training model")
    W = train(train_set, eta, epochs=epochs)
    print("\nCLASSIFY")
    tp, fp, fn = Counter(), Counter(), Counter()
    for ref_lang_vec, x in test_set:
        ref_lang = np.argmax(ref_lang_vec)

        hyp_lang = classify(W, x)
        if hyp_lang == ref_lang:
            tp[ref_lang] += 1
        else:
            fp[hyp_lang] += 1
            fn[ref_lang] += 1
    # Print metrics

    test_macro_f1 = macro_f1(tp, fp, fn)
    test_micro_f1 = micro_f1(tp, fp, fn)
    print(f"macro-averaged F1:\t\t{test_macro_f1:.3f}")
    print(f"micro-averaged F1:\t\t{test_micro_f1:.3f}")
    return test_macro_f1, test_micro_f1

### Putting it all together

Load the data:

In [75]:
train_observations = load_data("train.tsv")
test_observations = load_data("test.tsv")

### Inspecting the data

Before we actually train the classifier, it's important to look at your data, and check that any assumptions you're making about it are justified. It's always useful at this point to check basic things, like:
- How many instances of train and test data do you have?
- What labels are in your data, and do those match between the train and test splits?
- What is the class balance (i.e. how many instances of each class) in your dataset? Is it balanced or unbalanced?

Output a dictionary for the train and test data that maps each label to the count of instances that have that label, e.g.:
```
{"hausa": 4000, "indonesian":...}
```

Your dictionary should be sorted in descending order of occurrence of languages.

In [76]:
# this cell's output will be used for test 1.1
# your code here - train set
d = {"hausa": 0, "indonesian": 0, "manobo": 0, "nahuatl": 0, "swahili": 0, "tagalog": 0}
idxToName = ["hausa", "indonesian", "manobo", "nahuatl", "swahili", "tagalog"]
for k,v in train_observations:
  for n in idxToName:
    if k == n:
      d[k] += 1
print(d)

{'hausa': 91, 'indonesian': 86, 'manobo': 111, 'nahuatl': 90, 'swahili': 377, 'tagalog': 365}


In [77]:
# this cell's output will be used for test 1.2
# your code here - test set
d = {"hausa": 0, "indonesian": 0, "manobo": 0, "nahuatl": 0, "swahili": 0, "tagalog": 0}
idxToName = ["hausa", "indonesian", "manobo", "nahuatl", "swahili", "tagalog"]
for k,v in test_observations:
  for n in idxToName:
    if k == n:
      d[k] += 1
print(d)

{'hausa': 28, 'indonesian': 32, 'manobo': 34, 'nahuatl': 36, 'swahili': 142, 'tagalog': 121}


### Set hyperparameters and parameters

Before training the model, we have to set the learning rate $\eta$ and the order of the ngrams used in feature extraction.

In [78]:
eta = 0.0005  # Do not change this.
epochs = 4  # Do not change this.

In [79]:
order_of_ngrams = 2  # Change this.

### Running our training and evaluation loops

Now train your classifier, and evaluate it on the test set. Vary the number of ngrams, and observe how it changes train and test F1.

In [80]:
train_set, lang_map, feature_map = preprocess_training_observations(
    train_observations, n=order_of_ngrams
)
test_set = preprocess_test_observations(test_observations, feature_map, lang_map, n=order_of_ngrams)


random.seed(27)
test_macro_f1, test_micro_f1 = evaluate(train_set, test_set, eta, epochs=epochs)

1120 training observations.
393 test observations.
Training model
0.8803571428571428  is the micro precision 
 0.8803571428571428 is the micro recall
0.9299107142857143  is the micro precision 
 0.9299107142857143 is the micro recall
0.9467261904761904  is the micro precision 
 0.9467261904761904 is the micro recall
0.9564732142857143  is the micro precision 
 0.9564732142857143 is the micro recall

CLASSIFY
macro-averaged F1:		0.979
micro-averaged F1:		0.987


Print the tuple of the best values of `(macro_f1, micro_f1)` for the model evaluated on your test set while varying the order of the ngrams. What value of n produced the best result? Why might that be?

In [81]:
# the output of this cell will be used for test 2.1
print((test_macro_f1, test_micro_f1))
#the value of n that worked the best is 2-ngrams. This may be because these languages use a
#"japanese style" of writing where 2-letter characters prefixes are representative of meaning
#rather than traditional 1 character writing. But i dont think their text is romanized.
#So it could laso be because knowing what I know about swahili, there are several common
#2 character prefixes and suffixes that characterize the words.

(0.9793453311893895, 0.9872773536895675)


## Inspecting Classification Results

We've trained our classifier, and your final F1 should be pretty close to 1.0. Great job! But what does that mean for the languages you're actually classifying? Let's rewrite our evaluation code to allow us to look at our results instance-by-instance. For this, we're going to examine the results of a re-trained trigram classifier, trained for one epoch.

In [82]:
# Re-training a trigram classifier. Do not change this.

INSPECTION_NGRAMS = 3

train_set, lang_map, feature_map = preprocess_training_observations(
    train_observations, n=INSPECTION_NGRAMS
)
test_set = preprocess_test_observations(
    test_observations, feature_map, lang_map, n=INSPECTION_NGRAMS
)

random.seed(27)
W_inspect = train(train_set, eta, epochs=1)

## evaluate it as before. Check that this looks the same!
tp, fp, fn = Counter(), Counter(), Counter()
for ref_lang_vec, x in test_set:
    ref_lang = np.argmax(ref_lang_vec)

    hyp_lang = classify(W_inspect, x)
    if hyp_lang == ref_lang:
        tp[ref_lang] += 1
    else:
        fp[hyp_lang] += 1
        fn[ref_lang] += 1
# Print metrics

print(f"macro-averaged F1:\t\t{macro_f1(tp, fp, fn):.3f}")
print(f"micro-averaged F1:\t\t{micro_f1(tp, fp, fn):.3f}")

1120 training observations.
393 test observations.
0.8580357142857142  is the micro precision 
 0.8580357142857142 is the micro recall
macro-averaged F1:		0.949
micro-averaged F1:		0.962


### Writing a prediction function

Write a function that takes a list of observations, and produces a list of either class indices, or class names based on a parameter. This will allow you both to look at individual results from evaluating on an existing set of data (like the test set), but also for you to evaluate your classifier on new data (i.e. any string). This will involve vectorizing the list of documents, and classifying those vectors. Once that's complete, construct an inverse language mapping, from predicted indices to the language they represent.

In [83]:
def predict(
    documents: list[str],
    W: np.ndarray,
    feature_map: dict[str, int],
    lang_map: dict[str, np.ndarray],
    ngram_length: int,
    return_class_names=False,
):
    # your code here
    # a dictionary maps the string to a ohv.
    #what do i do with calssified feature vectors
    #init a list set return a list of the class names, otherwise you return whatever output form classified function
    #true => ["Indonesian", "...."]
    # false => just return the indices. [3,5,6...]
    idxToName = ["hausa", "indonesian", "manobo", "nahuatl", "swahili", "tagalog"]
    out = []
    for doc in documents:
      featureVector = vectorize_document(doc, feature_map, ngram_length)
      idx = classify(W, featureVector)
      if return_class_names == True:
        out = np.append(out, idxToName[idx])
      else:
        out = np.append(out, idx)
    return out

Now, for each instance in the test set, print a tuple of the actual label, then the predicted label.

In [84]:
# the output of this cell will be used for test 3.1
test_labels, test_documents = zip(*test_observations)
test_predictions = predict(
    test_documents,
    W_inspect,
    feature_map,
    lang_map,
    ngram_length=INSPECTION_NGRAMS,
    return_class_names=True,
)

print(list(zip(test_labels, test_predictions)))

[('swahili', 'swahili'), ('indonesian', 'indonesian'), ('tagalog', 'tagalog'), ('indonesian', 'indonesian'), ('hausa', 'hausa'), ('nahuatl', 'nahuatl'), ('hausa', 'hausa'), ('manobo', 'manobo'), ('swahili', 'swahili'), ('swahili', 'swahili'), ('tagalog', 'tagalog'), ('indonesian', 'indonesian'), ('tagalog', 'tagalog'), ('tagalog', 'tagalog'), ('swahili', 'swahili'), ('swahili', 'swahili'), ('hausa', 'swahili'), ('indonesian', 'indonesian'), ('manobo', 'manobo'), ('tagalog', 'tagalog'), ('swahili', 'swahili'), ('swahili', 'swahili'), ('manobo', 'manobo'), ('swahili', 'swahili'), ('manobo', 'manobo'), ('swahili', 'swahili'), ('tagalog', 'tagalog'), ('swahili', 'swahili'), ('swahili', 'swahili'), ('nahuatl', 'nahuatl'), ('nahuatl', 'nahuatl'), ('swahili', 'swahili'), ('swahili', 'swahili'), ('swahili', 'swahili'), ('tagalog', 'tagalog'), ('indonesian', 'indonesian'), ('nahuatl', 'nahuatl'), ('tagalog', 'tagalog'), ('tagalog', 'tagalog'), ('tagalog', 'tagalog'), ('tagalog', 'tagalog'), ('t

### Plotting a Confusion Matrix

 A _confusion matrix_ is a $k \times k$ matrix, where k is your number of classes, where the cell in position $(i,j)$ counts the number of instances that belong to class $i$ that were predicted to be in class $j$. The diagonal entries represent correct classifications; anything off of the diagonal represents an incorrect classification. The example below, from the [scikit learn documentation](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.ConfusionMatrixDisplay.html), shows a confusion matrix for a binary classification problem.

![A confusion matrix for a binary classification problem](confusion_matrix.png)


Confusion matrices can be useful to see what types of errors your classifier is making. If errors are concentrated into particular cells, it could indicate the kind of data that your classifier struggles with. Write a function to take a list of test observations, and output a numpy array that represents your classifier's confusion matrix.

In [85]:
def get_confusion_matrix(
    labels: list[str], predictions: list[str], lang_map: dict[str, np.ndarray]
) -> np.ndarray:
    # your code here
    #matrix should be a list of lists, K * K matrix, np array.
    #non diagonal are incorrect
    #columns are unordered, just use the order of the items in the labels
    #can use order of items in the lang map, first one is hausa.
    #output format may get no points
    #hausa weight is just the one row of the weights corresponding to hausa.
    #print(lang_map)
    K = len(lang_map)
    cnf = np.zeros((K,K))
    for i in range(0,len(labels)):
      lblIdx = np.argmax(lang_map[labels[i]])
      predIdx = np.argmax(lang_map[predictions[i]])
      cnf[lblIdx][predIdx] += 1
    return cnf

Now, print your confusion matrix.

In [86]:
# the output of this cell will be used for test 3.2
get_confusion_matrix(test_labels, test_predictions, lang_map)

array([[ 18.,   1.,   0.,   0.,   9.,   0.],
       [  0.,  31.,   0.,   0.,   0.,   1.],
       [  0.,   0.,  33.,   0.,   1.,   0.],
       [  0.,   0.,   0.,  34.,   2.,   0.],
       [  0.,   0.,   0.,   0., 142.,   0.],
       [  0.,   0.,   0.,   0.,   1., 120.]])

From your confusion matrix, how many Hausa examples are misclassified as Swahili? Print each of the misclassified documents on a new line.

In [87]:
# the output of this cell will be used for test 3.3
# your code here
labels = test_labels
predictions = test_predictions
for i in range(0,len(labels)):
      lblIdx = np.argmax(lang_map[labels[i]])
      predIdx = np.argmax(lang_map[predictions[i]])
      if lblIdx == 0 and predIdx == 4:
        print(test_documents[i])

Dalilin haka kuwa sun nuna cewa ayukan da shari'a take bukata na nan a rubuce a zuciyarsu. lamirinsu kuma na yi masu shaida, tunanainsu kuma, ko dai yana kashe su, ko kuma yana karesu.
Sai aka kawo kansa bisa tire, aka mika wa yarinyar, ta kuwa kai wa mahaifiyarta.
Ta wurin haka Yesu ya bada tabbatacce da kuma ingataccen alkwari.
Baku sani ba mu za mu yiwa mala'iku shari'a? Balle shari'ar al'amuran wannan rai?
Ya ce masu, "A rubuce yake, cewa Almasihu za ya sha wuya, zai tashi kuma daga matattu a rana ta uku.
Bayan shekara uku na je Urushalima don in san Kefas, na zauna da shi na kwana goma sha biyar.
Ga wani yin ayyukan iko, ga wani kuwa annabci. Ga wani kuwa an ba shi baiwar bambance ruhohi, ga wani harsuna daban daban, kuma ga wani fassarar harsuna.
Amma da hankalinsa ya dawo, ya ce, 'Barorin mahaifina su nawa ne da suke da abinci isasshe, amma ina nan a nan, ina mutuwa sabili da yunwa!
Wannan dabban da na gani yana kama da damisa, kafafunsa kuma kamar na beyar, bakinsa kuma kamar n

Can you formulate a hypothesis for why these Hausa examples are being misclassified as Swahili? Peruse the Wikipedia pages of the two languages, and inspect the data and features of the two languages. Consider reasons based in what you know about the languages, and about machine learning. Try to come up with 2-3 experiments you might run to validate your hypothesis.

### Examining Feature Weights

The classifier that we've built for softmax classification behaves in many ways like using several binary softmax classifiers stacked together. The $K \times N$ feature matrix can interpreted as a $1 \times N$ feature vector for each class. In this section, we'll examine the weights for a few of our classified languages. To get feature names out of these vectors, we'll construct an inverted feature map, that maps indices in the feature vectors back to n-grams. Then, extract the $1 \times N$ vector that corresponds to Hausa. Print the shape of the Hausa feature vector.

In [88]:
# the output of this cell will be used for test 4.1
# Invert the feature map
inverted_feature_map = {v: k for k, v in feature_map.items()}
#print(inverted_feature_map)
hausaV = W_inspect[0]
print(np.shape(hausaV))
# your code here

(7942,)


Now, let's find the features that are most strongly predictive of an instance being Hausa. From the Hausa feature vector, find the names of the features with the top-10 positive values. Print your results with a tuple of (feature name, feature_weight) on each line:
```
("aaa", 0.04597442104739769)
("bbb", 0.03454984682736487)
```

In [89]:
# the output of this cell will be used for test 4.2
# your code here
ind = np.argpartition(hausaV, -10)[-10:]
mjr = []
for idx in ind:
  mjr.append((inverted_feature_map[idx],hausaV[idx]))
out = sorted(mjr, key=lambda x:x[1])
out.reverse()
for item in out:
  print(item)

('da!', 0.063150564082901)
(' de', 0.054363893172131285)
('in!', 0.027868422071017727)
(' ye', 0.02487968026676279)
("ar'", 0.023277611165428837)
(' si', 0.02198093247915512)
(' be', 0.021048163450823363)
('ai,', 0.020038068668300366)
('ka!', 0.019952359788949974)
('ya!', 0.01917826911896685)


Now, for each language, print the top-10 features. Print the name of each language, and then a list of it's top-10 features on the following line, e.g.

```
hausa
[("aaa", 0.04597442104739769)...]
indonesian
[("aaa", 0.04597442104739769)...]
```

In [90]:
# the output of this cell will be used for test 4.3
# your code here
idxToName = ["hausa", "indonesian", "manobo", "nahuatl", "swahili", "tagalog"]
for i in range(0,len(idxToName)):
  v = W_inspect[i]
  ind = np.argpartition(v, -10)[-10:]
  print(idxToName[i])
  cmnftrs = []
  for idx in ind:
    cmnftrs.append((inverted_feature_map[idx],v[idx]))
  out = sorted(cmnftrs, key=lambda x:x[1])
  out.reverse()
  print(out)

hausa
[('da!', 0.063150564082901), (' de', 0.054363893172131285), ('in!', 0.027868422071017727), (' ye', 0.02487968026676279), ("ar'", 0.023277611165428837), (' si', 0.02198093247915512), (' be', 0.021048163450823363), ('ai,', 0.020038068668300366), ('ka!', 0.019952359788949974), ('ya!', 0.01917826911896685)]
indonesian
[('an!', 0.06804588397633496), (' mf', 0.06296123413453153), ('ani', 0.05154754483007454), ('bes', 0.03768695995865839), (' bi', 0.037192031587281224), (' kh', 0.03452062978681125), (' sh', 0.03324158839344721), ('meo', 0.033132192231201625), (' do', 0.03245654176502971), ('ah!', 0.030398224045560432)]
manobo
[(' tr', 0.107483603861862), ('to!', 0.10445031656438448), (' nu', 0.07028591690295224), ('no,', 0.05550510517116152), ('an!', 0.035445216508256695), (' oj', 0.035146648956816885), ('ow,', 0.03348412435308619), ('n u', 0.02977386180101167), ('dio', 0.028821441995230877), ('on,', 0.027656726512025165)]
nahuatl
[('tle', 0.0634978527218824), (' to', 0.0603224407223251

Choose one of the languages, and peruse its Wikipedia page or other reliable resources to learn a bit more about the language. Do the top 10 features in that language make sense given what you've learned about the structure of the language? Why or why not? Again, consider reasons stemming both from what you've learned about the language, as well as machine learning and multinomial logistic regression specifically.

### Extra Credit

Implement at least one of the experiments you devised to test your hypothesis regarding misclassification of Hausa as Swahili. In order to get credit you must submit not just the code and results, but also clearly describe your hypothesis, the experiment, and why the experiment is suitable to test your hypothesis. Full credit will be given to hypotheses and experiments that are well thought out, explained clearly and convincingly, and backed up with suitable evidence (computed or cited).

In [91]:
# Implement extra credit here.
# To try to decrease the probability of misclassification between hausa and indonesian, I attempted to find probable features in hausa that were improbable in swahili, and vice versa. This was done using the following code:
idxToName = ["hausa", "indonesian", "manobo", "nahuatl", "swahili", "tagalog"]
ngram_length = 3
diffArr = np.array([])
for i in range(0,len(W_inspect[0])):
  diffArr = np.append(diffArr, abs(W_inspect[0][i]-W_inspect[4][i]))
amax = np.argmax(diffArr)
ind = np.argpartition(diffArr, -10)[-10:]
#print(idxToName[i])
cmnftrs = []
for idx in ind:
  cmnftrs.append((inverted_feature_map[idx],diffArr[idx], idx))
print(cmnftrs)
print(len(diffArr), amax)
#print(W_inspect[0][7333], W_inspect[4][7333])

#After these are computed, we can change our classify function to look for these specific features when
#the probability of swahili and probability of hausa are similar
#this method may help in resolving knifes edge issues where the model comes "close" to classifying correctly,
#say use this method when the probability of hausa and swahili are within 5% of each other.

[(' ky', 0.0687249563112323, 426), ('an!', 0.06888746764311318, 2595), ('da!', 0.07662190642693825, 3072), ('a n', 0.08183085157583467, 2264), ('na!', 0.09365807851858221, 5463), ('a l', 0.08281409330134763, 2262), ('wa!', 0.13904290837497557, 7333), (' we', 0.11906075658844521, 541), (' nc', 0.08618779662358586, 455), (' de', 0.08409746274560283, 356)]
7942 7333
